In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.audio import *

In [3]:
sg_cfg= SpectrogramConfig(hop=480, n_mels=128, n_fft=960, top_db=80, f_min=20.0, f_max=22050)
config_split= AudioConfig(resample_to = 8000, remove_silence = "all", silence_padding=200, silence_threshold=20, sg_cfg=sg_cfg)

In [4]:
train_df = pd.read_csv(Path("./data/train.tsv"),sep="\t")

In [201]:
class DeepSpeechAudioList(AudioList):
    def __init__(self, items, path, config=AudioConfig(), context=9, **kwargs):
        super().__init__(items=items, path=path, config=config, **kwargs)
        self.context = context
        
    def get(self,i):
        one_spectro = self.open(self.items[i]).spectro
        return [one_spectro[:,:,i:i+(self.context*2+1)] for i in range(one_spectro.shape[-1]-(self.context*2+1))]

In [226]:
data = (DeepSpeechAudioList.from_folder("data/clips", config=config_split)#, processor=[rs])
        .use_partial_data(0.01, seed=42)
        .filter_by_func(lambda x: x.name in list(train_df.path))
        .split_by_rand_pct(0.2, seed=42)
        .label_from_func(lambda x: train_df[train_df.path==str(x).split("/")[-1]]["sentence"].iloc[0], label_cls=ItemList)
        .databunch()
       )

Preprocessing: Resampling to 8000


Preprocessing: Removing Silence


Preprocessing: Resampling to 8000


Preprocessing: Removing Silence


In [266]:
data.train_ds[0][0][0]

tensor([[[-47.1463, -47.1463, -47.1463,  ..., -47.1463, -47.1463, -47.1463],
         [-47.1463, -47.1463, -47.1463,  ...,  -7.9309, -17.1496, -13.9057],
         [-47.1463, -47.1463, -47.1463,  ..., -11.4937, -20.7124, -17.4685],
         ...,
         [-41.2216, -34.7202, -39.9796,  ...,  -2.3120,  -5.6158, -16.1828],
         [-43.3656, -42.7477, -41.8937,  ...,  -7.2488,  -7.0687, -19.9407],
         [-44.1028, -43.0013, -41.9343,  ..., -11.3914, -12.8795, -21.8022]]])

In [260]:
data.train_ds[0][1]

'Ridgway will never be eligible for parole.'

In [347]:
class DeepSpeech(nn.Module):
    def __init__(self, c=9):
        super(DeepSpeech, self).__init__()
        self.c = c
        self.h = torch.randn((2,128,2048))
        self.h1 = nn.Linear(19,2048)
        self.h2 = nn.Linear(2048,2048)
        self.h3 = nn.Linear(2048,2048)
        self.h4 = nn.RNN(2048,2048, nonlinearity="relu",bidirectional=True)
        self.h5 = nn.Linear(524288,29) #ct ∈ {a,b,c, . . . , z, space, apostrophe, blank}
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.stack(x,dim=1)
        x = self.h1(x).clamp(min=0, max=20)
        x = self.h2(x).clamp(min=0, max=20)
        x = self.h3(x).clamp(min=0, max=20)
        x,h = self.h4(x.squeeze(), self.h)
        self.h = h
        x = nn.Flatten()(x)
        x = self.h5(x).clamp(min=0, max=20)
        x = self.softmax(x)
        return x

In [348]:
tst = DeepSpeech(c=9)

In [349]:
torch.stack(data.train_ds[0][0],dim=1).shape

torch.Size([1, 28, 128, 19])

In [350]:
tst1 = tst(data.train_ds[0][0])#.argmax()

In [351]:
tst1.shape

torch.Size([28, 29])

In [353]:
tst1.argmax(dim=1)

tensor([20, 13, 13, 13, 20, 11,  7, 20,  5, 13,  5, 11,  8, 11, 11, 11, 13, 20,
        20, 13, 20, 13, 13,  3, 13, 28,  6,  7])

In [303]:
tst1[0].shape

torch.Size([128, 29])

In [29]:
tst1[1].shape

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [80]:
tst2 = nn.RNN(2048,2048,bidirectional=True, nonlinearity="relu")

In [81]:
tst2(tst1)

(tensor([[[0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0215, 0.0292],
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0215, 0.0292],
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0215, 0.0292],
          ...,
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0215, 0.0292],
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0215, 0.0292],
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0215, 0.0292]]],
        grad_fn=<CatBackward>),
 tensor([[[0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0000, 0.0094],
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0000, 0.0094],
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0000, 0.0094],
          ...,
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0000, 0.0094],
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0000, 0.0094],
          [0.0000, 0.0081, 0.0000,  ..., 0.0000, 0.0000, 0.0094]],
 
         [[0.0000, 0.0119, 0.0079,  ..., 0.0000, 0.0215, 0.0292],
          [0.0000, 0.0119, 0.0079,  ..., 0.0000, 0.0215, 0.0292],
         

In [52]:
tst2

(tensor([[[ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ...,  6.7068e-03,
           -2.0349e-05, -1.5341e-03],
          [ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ...,  6.7068e-03,
           -2.0349e-05, -1.5341e-03],
          [ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ...,  6.7068e-03,
           -2.0349e-05, -1.5341e-03],
          ...,
          [ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ...,  6.7068e-03,
           -2.0349e-05, -1.5341e-03],
          [ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ...,  6.7068e-03,
           -2.0349e-05, -1.5341e-03],
          [ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ...,  6.7068e-03,
           -2.0349e-05, -1.5341e-03]]], grad_fn=<CatBackward>),
 (tensor([[[ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ..., -8.3725e-03,
            -2.1176e-03, -8.3589e-03],
           [ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ..., -8.3725e-03,
            -2.1176e-03, -8.3589e-03],
           [ 2.3006e-03, -3.5222e-03,  5.7094e-03,  ..., -8.3725e-03,
            -2.1176e-03, -8.3

In [20]:
models.ResNet??

In [14]:
learn.data.train_ds[0][0]

AudioItem AudioItem 2.6 seconds (1 channels, 124800 samples @ 48000hz)